In [2]:
import joblib
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType, StringTensorType
import pandas as pd

load the scikit model

In [3]:
clf = joblib.load('../model.pkl')

some auxiliary function to help converting inputs to ONNX format

In [4]:
def convert_sklearn_onnx(clf, x_sample):
    inputs = convert_dataframe_schema(x_sample)
    onnx_model = convert_sklearn(clf, 'model_pipeline', inputs, target_opset=11)

    with open("model.onnx", "wb") as f:
        f.write(onnx_model.SerializeToString())

    return onnx_model


def convert_dataframe_schema(df, drop=None):
    inputs = []
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue
        if v == 'int64':
            t = Int64TensorType([None, 1])
        elif v == 'float64':
            t = FloatTensorType([None, 1])
        else:
            t = StringTensorType([None, 1])
        inputs.append((k, t))
    return inputs


def onnx_input(x):cl;o
    inputs = {c: x[c].values for c in x.columns}
    for k in inputs:
        inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))
    return inputs

In [20]:
x = pd.read_csv('../data/adult_test.csv').drop(['income'], axis=1)
x.to_csv('./prediction_batch.csv', index = False)
x['age']=x['age'].astype(int)
x['hours_per_week']=x['hours_per_week'].astype(int)
x['capital_gain']=x['capital_gain'].astype(int)
x['capital_loss']=x['capital_loss'].astype(int)

model conversion (needs an input example)

In [6]:
onnx_model = convert_sklearn_onnx(clf, x)

Cool model Viz tool that shows computational graph

In [7]:
from onnx.tools.net_drawer import GetPydotGraph, GetOpNodeProducer
pydot_graph = GetPydotGraph(onnx_model.graph, name=onnx_model.graph.name, rankdir="TP",
                            node_producer=GetOpNodeProducer("docstring"))
pydot_graph.write_dot("graph.dot")

In [8]:
import os
os.system('dot -O -Tpng graph.dot')

0

inference time (onnxruntime is quite lightweight)

In [9]:
import onnxruntime as rt

In [10]:
sess = rt.InferenceSession('./model.onnx')


onnx_outputs = sess.run(None, onnx_input(x))


In [12]:
onnx_outputs[0]

array(['>50K', '<=50K', '<=50K', ..., '<=50K', '<=50K', '>50K'],
      dtype=object)

In [16]:
pd.DataFrame(onnx_outputs[1]).to_csv('predictions.csv')